In [1]:
import pandas as pd
import numpy as np
from scipy import stats


In [3]:
df = pd.read_csv("C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/content_engagement_churn_analytics/data/processed/engagement_features.csv")
df.head()

,user_id,subscription_plan,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend,engagement_score
0,U00001,Basic,1,122.327536,4.547826,71.764339,122.122885,122.184636,14.140580,0.143272,28.766061
1,U00002,Basic,1,121.375000,4.601562,63.786368,121.955407,121.521901,14.363281,-0.337561,30.794558
2,U00003,Basic,1,124.544828,4.583908,67.809904,123.947411,122.452469,14.261494,0.251152,30.850132
3,U00004,Free,1,128.353247,4.927273,69.014366,128.166358,127.670915,5.371558,-0.169271,32.115171
4,U00005,Basic,1,121.992167,4.738903,63.305509,121.290066,120.139129,14.542428,0.096485,31.226885


In [5]:
df["churn_flag"].value_counts()


churn_flag
1    1196
0       4
Name: count, dtype: int64

In [7]:
df.groupby("churn_flag")["engagement_score"].mean()
df.head()

,user_id,subscription_plan,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend,engagement_score
0,U00001,Basic,1,122.327536,4.547826,71.764339,122.122885,122.184636,14.140580,0.143272,28.766061
1,U00002,Basic,1,121.375000,4.601562,63.786368,121.955407,121.521901,14.363281,-0.337561,30.794558
2,U00003,Basic,1,124.544828,4.583908,67.809904,123.947411,122.452469,14.261494,0.251152,30.850132
3,U00004,Free,1,128.353247,4.927273,69.014366,128.166358,127.670915,5.371558,-0.169271,32.115171
4,U00005,Basic,1,121.992167,4.738903,63.305509,121.290066,120.139129,14.542428,0.096485,31.226885


In [9]:
churned = df[df["churn_flag"] == 1]["engagement_score"]
active = df[df["churn_flag"] == 0]["engagement_score"]

t_stat, p_value = stats.ttest_ind(
    churned,
    active,
    equal_var=False
)

t_stat, p_value


(-2.665343920160452, 0.07462467883665222)

In [11]:
if p_value < 0.05:
    print("Churn is NOT random — engagement significantly impacts churn")
else:
    print("No significant engagement difference")


No significant engagement difference


In [13]:
df.groupby("subscription_plan")["revenue_30d"].mean()


subscription_plan
Basic      14.146916
Free        5.127675
Premium    22.787306
Name: revenue_30d, dtype: float64

In [15]:
stats.pearsonr(
    df["engagement_score"],
    df["revenue_30d"]
)


PearsonRResult(statistic=0.017652239772141094, pvalue=0.541264563751271)

In [19]:
df["churn_risk_score"] = (
    0.4 * (df["engagement_score"].max() - df["engagement_score"]) +
    0.3 * abs(df["engagement_trend"].fillna(0)) +
    0.3 * df["engagement_volatility"].fillna(0)
)
df.head()

,user_id,subscription_plan,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend,engagement_score,churn_risk_score
0,U00001,Basic,1,122.327536,4.547826,71.764339,122.122885,122.184636,14.140580,0.143272,28.766061,24.727127
1,U00002,Basic,1,121.375000,4.601562,63.786368,121.955407,121.521901,14.363281,-0.337561,30.794558,21.580623
2,U00003,Basic,1,124.544828,4.583908,67.809904,123.947411,122.452469,14.261494,0.251152,30.850132,22.739532
3,U00004,Free,1,128.353247,4.927273,69.014366,128.166358,127.670915,5.371558,-0.169271,32.115171,22.570291
4,U00005,Basic,1,121.992167,4.738903,63.305509,121.290066,120.139129,14.542428,0.096485,31.226885,21.191112


In [21]:
df["churn_risk_segment"] = pd.qcut(
    df["churn_risk_score"],
    q=3,
    labels=["Low Risk", "Medium Risk", "High Risk"]
)


In [23]:
df["churn_risk_segment"].value_counts()


churn_risk_segment
Low Risk       400
Medium Risk    400
High Risk      400
Name: count, dtype: int64

In [25]:
pd.crosstab(
    df["subscription_plan"],
    df["churn_risk_segment"],
    normalize="index"
)


churn_risk_segment,Low Risk,Medium Risk,High Risk
subscription_plan,,,
Basic,0.306265,0.368910,0.324826
Free,0.336431,0.312268,0.351301
Premium,0.376623,0.316017,0.307359


In [27]:
df.to_csv(
    "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/content_engagement_churn_analytics/data/processed/churn_monetization_analysis.csv",
    index=False
)
